In [41]:
import re
import time
import requests

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

import pymorphy3 # работа с русским языком, pymorphy3

import texttable as tt

In [43]:
# рассмотрим данные прогноза погоды в село Самбург
# lat=66.80035786146756 (Широта)
# lng=78.38975066524623 (Долгота)
url = 'https://pogoda1.ru/samburg/arkhiv/'
page = requests.get(url)
bs = BeautifulSoup(page.text, 'html.parser')

# Формируем список диапозона даты, где имеются архивные данные
year = bs.find('select', {'class': 'select-archive-year'}).find_all('option')
month = bs.find('select', {'class': 'select-archive-month'}).find_all('option')

def month_translate(month):
  month = month.lower()
  if month == "январь":
    return "january"
  elif month == "февраль":
    return "february"
  elif month == "март":
    return "march"
  elif month == "апрель":
    return "april"
  elif month == "май":
    return "may"
  elif month == "июнь":
    return "june"
  elif month == "июль":
    return "july"
  elif month == "август":
    return "august"
  elif month == "сентября":
    return "september"
  elif month == "октябрь":
    return "october"
  elif month == "ноябрь":
    return "november"
  elif month == "декабрь":
    return "december"

base = []
for i, val_year in enumerate(year):
  for j, val_month in enumerate(month):
    temp_0 = []
    if j > 0:
      temp_0.append(int(val_year.text))
      temp_0.append(str(month_translate(val_month.text)).lower())
      base.append(temp_0)

# Формируем список URL адресов с данными
url_parse = 'https://pogoda1.ru/samburg/{month}-{year}/'

for i, val in enumerate(base):
  parse = url_parse.format(month=val[1], year=val[0])
  base[i].append(parse)

df = pd.DataFrame(base, columns=['Год', 'Месяц', 'URL'])
df


,Год,Месяц,URL
0,2017,january,https://pogoda1.ru/samburg/january-2017/
1,2017,february,https://pogoda1.ru/samburg/february-2017/
2,2017,march,https://pogoda1.ru/samburg/march-2017/
3,2017,april,https://pogoda1.ru/samburg/april-2017/
4,2017,may,https://pogoda1.ru/samburg/may-2017/
...,...,...,...
79,2023,august,https://pogoda1.ru/samburg/august-2023/
80,2023,none,https://pogoda1.ru/samburg/none-2023/
81,2023,october,https://pogoda1.ru/samburg/october-2023/
82,2023,november,https://pogoda1.ru/samburg/november-2023/


In [2]:
# рассмотрим данные прогноза погоды в село Самбург
# lat=66.80035786146756 (Широта)
# lng=78.38975066524623 (Долгота)

url = 'https://yandex.ru/pogoda/month/january?lat={latitude}&lon={lontitude}'
url = url.format(latitude=66.80035786146756, lontitude=78.38975066524623)
print(url)
page = requests.get(url)
bs = BeautifulSoup(page.text, 'html.parser')
print(bs)

https://yandex.ru/pogoda/month/january?lat=66.80035786146756&lon=78.38975066524623
<!DOCTYPE html>
<html class="i-ua_js_no i-ua_css_standard" lang="ru"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><title>Погода в январе. Ямало-Ненецкий автономный округ: Яндекс Погода </title><script nonce="1699001223439457-5006288813322167551-balancer-l7leveler-kubr-yp-vla-57-BAL">;(function(d,e,c,r){e=d.documentElement;c="className";r="replace";e[c]=e[c][r]("i-ua_js_no","i-ua_js_yes");if(d.compatMode!="CSS1Compat")e[c]=e[c][r]("i-ua_css_standart","i-ua_css_quirks")})(document);;(function(d,e,c,n,w,v,f){e=d.documentElement;c="className";n="createElementNS";f="firstChild";w="http://www.w3.org/2000/svg";e[c]+=" i-ua_svg_"+(!!d[n]&&!!d[n](w,"svg").createSVGRect?"yes":"no");v=d.createElement("div");v.innerHTML="<svg/>";e[c]+=" i-ua_inlinesvg_"+((v[f]&&v[f].namespaceURI)==w?"yes":"no");})(document);;(function(d,e,p){e=d.documentElement;p="placeholder";e.className+=" i-u

In [3]:
# информация о погоде за Январь 2023

morph = pymorphy3.MorphAnalyzer(lang='ru')

january = []
temp_january = bs.find('table', {'class': 'climate-calendar'}).find_all('td', {'class': 'climate-calendar__cell'})
for i, val in enumerate(temp_january):
  temp = []
  if val.find('div', {'class': 'climate-calendar-day_colorless_yes'}):
    continue
  temp.append(int(val.find('div', {'class': 'climate-calendar-day__detailed-container-center'}).find_next('h6').text.split(',', 1)[0].split(' ', 1)[0]))
  temp.append(str(morph.parse(val.find('div', {'class': 'climate-calendar-day__detailed-container-center'}).find_next('h6').text.split(',', 1)[0].split(' ', 1)[1])[0].normal_form))
  temp.append(2023)
  temp.append(str(val.find('div', {'class': 'climate-calendar-day__detailed-container-center'}).find_next('h6').text.split(',', 1)[1]))
  temp.append(int(val.find('div', {'class': 'temp climate-calendar-day__detailed-basic-temp-day'}).find('span').text.replace('−', '-')))
  temp.append(int(val.find('div', {'class': 'temp climate-calendar-day__detailed-basic-temp-night'}).find('span').text.replace('−', '-')))
  temp.append(int(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[0].text.split(' ', 1)[0]))
  temp.append(int(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[1].text.split('%', 1)[0]) / 100)
  temp.append(float(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[2].find('div').text))
  temp.append(str(val.find_all('td', {'class': 'climate-calendar-day__detailed-data-table-cell climate-calendar-day__detailed-data-table-cell_value_yes'})[2].find('abbr').text))
  january.append(temp)
january

[[1, 'январь', 2023, ' вс', -21, -26, 760, 0.76, 5.3, 'ЮВ'],
 [2, 'январь', 2023, ' пн', -22, -22, 763, 0.79, 5.3, 'З'],
 [3, 'январь', 2023, ' вт', -21, -22, 764, 0.79, 5.8, 'Ю'],
 [4, 'январь', 2023, ' ср', -17, -20, 762, 0.81, 6.5, 'Ю'],
 [5, 'январь', 2023, ' чт', -19, -22, 764, 0.8, 6.0, 'Ю'],
 [6, 'январь', 2023, ' пт', -20, -22, 766, 0.79, 5.5, 'Ю'],
 [7, 'январь', 2023, ' сб', -19, -21, 766, 0.76, 4.8, 'Ю'],
 [8, 'январь', 2023, ' вс', -20, -22, 763, 0.76, 5.3, 'ЮВ'],
 [9, 'январь', 2023, ' пн', -22, -22, 761, 0.77, 5.5, 'Ю'],
 [10, 'январь', 2023, ' вт', -21, -24, 760, 0.77, 5.8, 'Ю'],
 [11, 'январь', 2023, ' ср', -24, -24, 762, 0.76, 5.1, 'Ю'],
 [12, 'январь', 2023, ' чт', -21, -21, 758, 0.77, 6.5, 'Ю'],
 [13, 'январь', 2023, ' пт', -19, -22, 756, 0.8, 6.4, 'Ю'],
 [14, 'январь', 2023, ' сб', -20, -20, 757, 0.79, 5.4, 'ЮВ'],
 [15, 'январь', 2023, ' вс', -17, -20, 759, 0.8, 4.9, 'Ю'],
 [16, 'январь', 2023, ' пн', -20, -21, 763, 0.8, 4.5, 'Ю'],
 [17, 'январь', 2023, ' вт', -19, 

In [62]:
# Датафрейм из массива
df = pd.DataFrame(january, columns=['День', 'Месяц', 'Год', 'День недели', 'Температура днем', 'Температура ночью', 'Давление (мм рт. ст.)', 'Влажность (%)', 'Скорость ветра', 'Направление ветра'])
df

,День,Месяц,Год,День недели,Температура днем,Температура ночью,Давление (мм рт. ст.),Влажность (%),Скорость ветра,Направление ветра
0,1,январь,2023,вс,-11,-16,746,0.85,5.5,ЮЗ
1,2,январь,2023,пн,-15,-16,748,0.85,4.5,Ю
2,3,январь,2023,вт,-16,-16,747,0.82,5.5,Ю
3,4,январь,2023,ср,-14,-15,747,0.82,4.8,ЮЗ
4,5,январь,2023,чт,-14,-14,749,0.83,4.6,ЮЗ
5,6,январь,2023,пт,-12,-13,750,0.86,5.3,ЮЗ
6,7,январь,2023,сб,-12,-14,751,0.86,4.1,ЮЗ
7,8,январь,2023,вс,-14,-17,753,0.82,4.5,ЮЗ
8,9,январь,2023,пн,-14,-15,751,0.80,4.6,Ю
9,10,январь,2023,вт,-12,-14,748,0.83,4.6,ЮЗ


In [63]:
# cоздаем объект Texttable
tab = tt.Texttable()

# устанавливаем стили таблицы
tab.set_cols_align(['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c'])
tab.header(['День', 'Месяц', 'Год', 'День недели', 'Температура днем', 'Температура ночью', 'Давление (мм рт. ст.)', 'Влажность (%)', 'Скорость ветра', 'Направление ветра'])

# преобразуем DataFrame в список списков (двумерный список)
data_list = df.values.tolist()

# добавляем данные в таблицу
for row in data_list:
  tab.add_row(row)

# получаем отформатированную таблицу в виде строки
table_string = tab.draw()

# выводим таблицу на экран
print(table_string)

+------+--------+------+-------+-------+-------+-------+-------+-------+-------+
| День | Месяц  | Год  | День  | Темпе | Темпе | Давле | Влажн | Скоро | Напра |
|      |        |      | недел | ратур | ратур |  ние  | ость  |  сть  | влени |
|      |        |      |   и   |   а   |   а   |  (мм  |  (%)  | ветра |   е   |
|      |        |      |       | днем  | ночью |  рт.  |       |       | ветра |
|      |        |      |       |       |       | ст.)  |       |       |       |
+======+========+======+=======+=======+=======+=======+=======+=======+=======+
|  1   | январь | 2023 |   вс  |  -11  |  -16  |  746  | 0.850 | 5.500 |  ЮЗ   |
+------+--------+------+-------+-------+-------+-------+-------+-------+-------+
|  2   | январь | 2023 |   пн  |  -15  |  -16  |  748  | 0.850 | 4.500 |   Ю   |
+------+--------+------+-------+-------+-------+-------+-------+-------+-------+
|  3   | январь | 2023 |   вт  |  -16  |  -16  |  747  | 0.820 | 5.500 |   Ю   |
+------+--------+------+----